In [ ]:
import os
import json
import random
import numpy as np
import pandas as pd
import commentjson
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModel,
    get_linear_schedule_with_warmup,
    AutoModelForTokenClassification
)
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score, classification_report
from seqeval.metrics import classification_report as seqeval_report
from tqdm.auto import tqdm

# 시드 고정
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True

set_seed(42)

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

# 한국어 특화 모델과 토크나이저 설정
MODEL_NAME = "klue/roberta-base"  # 한국어 특화 모델
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
MAX_LENGTH = 128
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 2e-5
WARMUP_RATIO = 0.1

with open('integrated_data.jsonc', 'r', encoding='utf-8') as f:
        example_data = commentjson.load(f)



# NER 태그 설정
ner_tags = ["O", "B-BOOK", "I-BOOK", "B-AUTHOR", "I-AUTHOR"]
tag2id = {tag: idx for idx, tag in enumerate(ner_tags)}
id2tag = {idx: tag for idx, tag in enumerate(ner_tags)}

# Intent 라벨 설정
intent_labels = ["도서검색", "작가검색"]
intent2id = {intent: idx for idx, intent in enumerate(intent_labels)}
id2intent = {idx: intent for idx, intent in enumerate(intent_labels)}

def align_entity_with_tokens(text, entities, tokenizer):
    """
    원본 텍스트와 엔티티 위치를 토큰화와 일치시키는 함수
    """
    aligned_entities = []
    
    for entity in entities:
        entity_text = text[entity["start"]:entity["end"]]
        entity_tokens = tokenizer.tokenize(entity_text)
        
        # 전체 텍스트의 토큰화 결과에서 엔티티 시작 위치 찾기
        prefix_text = text[:entity["start"]]
        prefix_tokens = tokenizer.tokenize(prefix_text)
        
        token_start = len(prefix_tokens)
        token_end = token_start + len(entity_tokens) - 1
        
        aligned_entities.append({
            "type": entity["type"],
            "token_start": token_start,
            "token_end": token_end,
            "start": entity["start"],
            "end": entity["end"]
        })
    
    return aligned_entities


# 데이터셋 클래스 정의 (한국어 특화)
class KoreanLibraryDataset(Dataset):
    def __init__(self, data, tokenizer, max_len, tag2id, intent2id):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.tag2id = tag2id
        self.intent2id = intent2id
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        text = item["text"]
        intent = item["intent"]
        entities = item["entities"]
        
        # 토큰화 전에 원본 텍스트와 엔티티 위치 정렬
        aligned_entities = align_entity_with_tokens(text, entities, self.tokenizer)
        
        # 토큰화 (special_tokens 포함)
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        
        input_ids = encoding["input_ids"].squeeze()
        attention_mask = encoding["attention_mask"].squeeze()
        
        # NER 레이블 초기화 (모두 O 태그)
        labels = torch.ones(self.max_len, dtype=torch.long) * self.tag2id["O"]
        
        # CLS 토큰 위치 계산
        cls_token_id = self.tokenizer.cls_token_id
        sep_token_id = self.tokenizer.sep_token_id
        
        # CLS 토큰 위치 찾기 (일반적으로 0)
        cls_position = 0
        
        # 각 엔티티에 대해 BIO 태그 적용 (special token 고려)
        for entity in aligned_entities:
            entity_type = entity["type"]
            # CLS 토큰 때문에 오프셋 +1
            token_start = entity["token_start"] + 1
            token_end = entity["token_end"] + 1
            
            # 시작 토큰에 B- 태그 적용
            if token_start < self.max_len:
                labels[token_start] = self.tag2id[f"B-{entity_type}"]
            
            # 나머지 토큰에 I- 태그 적용
            for i in range(token_start + 1, token_end + 1):
                if i < self.max_len:
                    labels[i] = self.tag2id[f"I-{entity_type}"]
        
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "intent_label": torch.tensor(intent, dtype=torch.long),
            "ner_labels": labels
        }

# 데이터셋 생성
train_data = example_data  # 실제로는 train/valid로 나눠야 함
valid_data = example_data[:3]  # 예시용 간단한 검증 세트

train_dataset = KoreanLibraryDataset(train_data, tokenizer, MAX_LENGTH, tag2id, intent2id)
valid_dataset = KoreanLibraryDataset(valid_data, tokenizer, MAX_LENGTH, tag2id, intent2id)

# 데이터로더 생성
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)

# 통합 모델 정의 (Intent 분류와 NER을 동시에 수행)
class JointIntentNERModel(nn.Module):
    def __init__(self, model_name, num_intents, num_ner_tags, dropout_prob=0.1):
        super(JointIntentNERModel, self).__init__()
        self.roberta = AutoModel.from_pretrained(model_name)
        
        # Intent 분류를 위한 레이어
        self.intent_classifier = nn.Sequential(
            nn.Linear(self.roberta.config.hidden_size, self.roberta.config.hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(self.roberta.config.hidden_size, num_intents)
        )
        
        # NER을 위한 레이어
        self.ner_classifier = nn.Sequential(
            nn.Linear(self.roberta.config.hidden_size, self.roberta.config.hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(self.roberta.config.hidden_size, num_ner_tags)
        )
    
    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        pooled_output = sequence_output[:, 0, :]  # [CLS] 토큰 임베딩 사용
        
        # Intent 분류
        intent_logits = self.intent_classifier(pooled_output)
        
        # NER 태그 분류
        ner_logits = self.ner_classifier(sequence_output)
        
        return intent_logits, ner_logits

# 모델 초기화
model = JointIntentNERModel(
    model_name=MODEL_NAME, 
    num_intents=len(intent_labels), 
    num_ner_tags=len(ner_tags)
)
model.to(device)

# 옵티마이저 및 스케줄러 설정
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_dataloader) * EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=warmup_steps, 
    num_training_steps=total_steps
)

# 손실 함수
intent_criterion = nn.CrossEntropyLoss()
ner_criterion = nn.CrossEntropyLoss(ignore_index=-100)  # padding된 부분은 손실 계산에서 제외



# 모델 저장 함수
def save_model(model, tokenizer, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 모델 저장
    torch.save(model.state_dict(), os.path.join(output_dir, "model.pt"))
    
    # 토크나이저 저장
    tokenizer.save_pretrained(output_dir)
    
    # 모델 설정 저장
    config = {
        "intent_labels": intent_labels,
        "ner_tags": ner_tags,
        "max_length": MAX_LENGTH
    }
    
    with open(os.path.join(output_dir, "config.json"), "w", encoding="utf-8") as f:
        json.dump(config, f, ensure_ascii=False, indent=2)

def train_and_evaluate(model, train_dataloader, valid_dataloader, optimizer, scheduler, 
                      device, num_epochs, output_dir):
    best_f1 = 0
    
    # 전체 훈련 과정을 위한 단일 진행 표시줄
    total_steps = num_epochs * (len(train_dataloader) + len(valid_dataloader) + 2)  # +2는 에포크별 요약 단계
    main_progress_bar = tqdm(total=total_steps, desc="훈련 진행률")
    
    for epoch in range(num_epochs):
        # 에포크 정보 업데이트 (tqdm에서 표시)
        main_progress_bar.set_description(f"에포크 {epoch+1}/{num_epochs}")
        
        # 훈련 단계
        model.train()
        total_intent_loss = 0
        total_ner_loss = 0
        intent_preds, intent_labels = [], []
        ner_preds, ner_true = [], []
        
        for batch in train_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            intent_label = batch["intent_label"].to(device)
            ner_labels = batch["ner_labels"].to(device)
            
            # 모델 예측
            intent_logits, ner_logits = model(input_ids, attention_mask)
            
            # Intent 손실 계산
            intent_loss = intent_criterion(intent_logits, intent_label)
            
            # NER 손실 계산 (모든 토큰에 대해)
            ner_loss = ner_criterion(ner_logits.view(-1, len(ner_tags)), ner_labels.view(-1))
            
            # 전체 손실 (NER 손실과 Intent 손실의 가중합)
            loss = intent_loss + ner_loss
            
            # 역전파 및 옵티마이저 스텝
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # 그래디언트 클리핑
            optimizer.step()
            scheduler.step()
            
            # 손실 누적
            total_intent_loss += intent_loss.item()
            total_ner_loss += ner_loss.item()
            
            # Intent 예측값 및 실제값 저장
            intent_preds.extend(torch.argmax(intent_logits, dim=1).cpu().numpy())
            intent_labels.extend(intent_label.cpu().numpy())
            
            # NER 예측값 및 실제값 저장
            ner_pred = torch.argmax(ner_logits, dim=2).cpu().numpy()
            ner_true_labels = ner_labels.cpu().numpy()
            
            # attention_mask를 사용하여 패딩된 부분을 제외하고 예측 및 실제 NER 태그 저장
            for i, mask in enumerate(attention_mask.cpu().numpy()):
                pred_tags = [id2tag[p] for j, p in enumerate(ner_pred[i]) if mask[j] == 1]
                true_tags = [id2tag[t] if t != -100 else "O" for j, t in enumerate(ner_true_labels[i]) if mask[j] == 1]
                ner_preds.append(pred_tags)
                ner_true.append(true_tags)
            
            # 진행 상황 업데이트
            train_intent_loss = total_intent_loss / len(train_dataloader) if len(train_dataloader) > 0 else 0
            train_ner_loss = total_ner_loss / len(train_dataloader) if len(train_dataloader) > 0 else 0
            
            main_progress_bar.set_postfix({
                '단계': '훈련',
                'intent_loss': f"{train_intent_loss:.4f}",
                'ner_loss': f"{train_ner_loss:.4f}"
            })
            main_progress_bar.update(1)  # 프로그레스 바 1단계 진행
        
        # 훈련 평가 지표 계산
        train_intent_accuracy = accuracy_score(intent_labels, intent_preds)
        train_intent_f1 = f1_score(intent_labels, intent_preds, average='weighted')
        
        # NER 평가 지표 계산 (seqeval 사용)
        train_ner_report = seqeval_report(ner_true, ner_preds, output_dict=True, zero_division=0)
        train_ner_f1 = train_ner_report['weighted avg']['f1-score']
        
        # 평가 단계 
        model.eval()
        total_intent_loss = 0
        total_ner_loss = 0
        intent_preds, intent_labels_list = [], []
        ner_preds, ner_true = [], []
        
        with torch.no_grad():
            for batch in valid_dataloader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                intent_label = batch["intent_label"].to(device)
                ner_labels = batch["ner_labels"].to(device)
                
                # 모델 예측
                intent_logits, ner_logits = model(input_ids, attention_mask)
                
                # Intent 손실 계산
                intent_loss = intent_criterion(intent_logits, intent_label)
                
                # NER 손실 계산
                ner_loss = ner_criterion(ner_logits.view(-1, len(ner_tags)), ner_labels.view(-1))
                
                # 손실 누적
                total_intent_loss += intent_loss.item()
                total_ner_loss += ner_loss.item()
                
                # Intent 예측값 및 실제값 저장
                intent_preds.extend(torch.argmax(intent_logits, dim=1).cpu().numpy())
                intent_labels_list.extend(intent_label.cpu().numpy())
                
                # NER 예측값 및 실제값 저장
                ner_pred = torch.argmax(ner_logits, dim=2).cpu().numpy()
                ner_true_labels = ner_labels.cpu().numpy()
                
                # attention_mask를 사용하여 패딩된 부분을 제외하고 예측 및 실제 NER 태그 저장
                for i, mask in enumerate(attention_mask.cpu().numpy()):
                    pred_tags = [id2tag[p] for j, p in enumerate(ner_pred[i]) if mask[j] == 1]
                    true_tags = [id2tag[t] if t != -100 else "O" for j, t in enumerate(ner_true_labels[i]) if mask[j] == 1]
                    ner_preds.append(pred_tags)
                    ner_true.append(true_tags)
                
                # 진행 상황 업데이트 
                val_intent_loss = total_intent_loss / len(valid_dataloader) if len(valid_dataloader) > 0 else 0
                val_ner_loss = total_ner_loss / len(valid_dataloader) if len(valid_dataloader) > 0 else 0
                
                main_progress_bar.set_postfix({
                    '단계': '평가',
                    'intent_loss': f"{val_intent_loss:.4f}",
                    'ner_loss': f"{val_ner_loss:.4f}"
                })
                main_progress_bar.update(1)  # 프로그레스 바 1단계 진행
        
        # Intent 평가 지표 계산
        val_intent_accuracy = accuracy_score(intent_labels_list, intent_preds)
        val_intent_f1 = f1_score(intent_labels_list, intent_preds, average='weighted')
        
        # 데이터셋에 실제로 존재하는 클래스 확인 (동적으로 처리)
        unique_labels = sorted(set(intent_labels_list))
        used_target_names = [id2intent[i] for i in unique_labels]
        
        # 실제 존재하는 클래스에 대해서만 classification_report 생성
        intent_report = classification_report(
            intent_labels_list, 
            intent_preds,
            target_names=used_target_names, 
            output_dict=True,
            zero_division=0
        )
        
        # NER 평가 지표 계산
        ner_report = seqeval_report(ner_true, ner_preds, output_dict=True, zero_division=0)
        val_ner_f1 = ner_report['weighted avg']['f1-score']
        
        # 에포크 결과 요약 
        avg_f1 = (val_intent_f1 + val_ner_f1) / 2
        
        # 진행바에 에포크 결과 표시
        main_progress_bar.set_postfix({
            'Ep': f"{epoch+1}/{num_epochs}",
            'intent_acc': f"{val_intent_accuracy:.4f}",
            'avg_f1': f"{avg_f1:.4f}"
        })
        main_progress_bar.update(1)  # 에포크 요약 단계 업데이트
        
        # 최고 성능 모델 저장
        if avg_f1 > best_f1:
            best_f1 = avg_f1
            save_model(model, tokenizer, output_dir)
            main_progress_bar.set_postfix({
                'Ep': f"{epoch+1}/{num_epochs}",
                'best_avg_f1': f"{avg_f1:.4f}",
                'saved': 'Y'
            })
        
        main_progress_bar.update(1)  # 모델 저장 단계 업데이트
    
    main_progress_bar.close()
    return best_f1

# 훈련 실행
best_f1 = 0
output_dir = "./korean_library_chatbot_model"
logging_steps = 10  # 10배치마다 진행 상황 업데이트

# 훈련 시작
print("\n훈련 시작...")
best_f1 = train_and_evaluate(
    model=model,
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    num_epochs=EPOCHS,
    output_dir=output_dir
)

print(f"\n훈련 완료! 최종 모델이 {output_dir}에 저장되었습니다. (최고 평균 F1: {best_f1:.4f})")

# 실제 텍스트에서 엔티티를 추출하는 함수 개선
def extract_entities_from_tokens(text, token_predictions, offset_mapping, id2tag):
    entities = []
    current_entity = None
    
    for i, (pred, (start, end)) in enumerate(zip(token_predictions, offset_mapping)):
        if start == end:  # 특수 토큰 건너뛰기
            continue
            
        tag = id2tag[pred]
        
        if tag.startswith("B-"):  # 엔티티 시작
            # 이전 엔티티가 있으면 저장
            if current_entity is not None:
                entities.append(current_entity)
            
            entity_type = tag[2:]  # "B-" 제거
            current_entity = {
                "type": entity_type,
                "start": start,
                "end": end,
                "text": text[start:end]
            }
        
        elif tag.startswith("I-") and current_entity is not None:
            # 이전 엔티티와 같은 타입인 경우만 확장
            if current_entity["type"] == tag[2:]:
                current_entity["end"] = end
                current_entity["text"] = text[current_entity["start"]:end]
        
        elif tag == "O":  # 엔티티 종료
            if current_entity is not None:
                entities.append(current_entity)
                current_entity = None
    
    # 마지막 엔티티 처리
    if current_entity is not None:
        entities.append(current_entity)
    
    # 중복 엔티티 제거 (더 긴 엔티티 우선)
    filtered_entities = []
    for entity in sorted(entities, key=lambda e: len(e["text"]), reverse=True):
        # 이미 포함된 엔티티인지 확인
        is_contained = False
        for filtered in filtered_entities:
            if (entity["start"] >= filtered["start"] and 
                entity["end"] <= filtered["end"] and
                entity["type"] == filtered["type"]):
                is_contained = True
                break
        
        if not is_contained:
            filtered_entities.append(entity)
    
    return filtered_entities

# 모델 추론 함수 개선
def predict_intent_and_entities(text, model, tokenizer, id2intent, id2tag):
    model.eval()
    
    # 토큰화 (오프셋 매핑 포함)
    encoding = tokenizer(
        text,
        max_length=MAX_LENGTH,
        padding="max_length",
        truncation=True,
        return_offsets_mapping=True,
        return_tensors="pt"
    )
    
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)
    offset_mapping = encoding["offset_mapping"].squeeze().cpu().numpy()
    
    with torch.no_grad():
        intent_logits, ner_logits = model(input_ids, attention_mask)
        
        # Intent 예측
        intent_pred = torch.argmax(intent_logits, dim=1).cpu().numpy()[0]
        intent_name = id2intent[intent_pred]
        
        # NER 예측
        ner_pred = torch.argmax(ner_logits, dim=2).cpu().numpy()[0]
        
        # 토큰화된 결과 확인 (디버깅용)
        print("\n토큰화 결과:")
        tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().numpy())
        for i, (token, pred) in enumerate(zip(tokens, ner_pred)):
            if attention_mask[0][i] == 1:  # 패딩이 아닌 토큰만
                print(f"  {token}: {id2tag[pred]}")
        
        # 실제 텍스트 범위에 맞게 엔티티 추출
        entities = extract_entities_from_tokens(text, ner_pred, offset_mapping, id2tag)
    
    return {
        "intent": intent_name,
        "entities": entities
    }
def postprocess_prediction(text, prediction, common_authors_set=None, common_books_set=None):
    current_intent = prediction["intent"]
    original_model_intent = prediction["intent"] 
    entities = prediction["entities"]
    
    has_author_entity = any(e["type"] == "AUTHOR" for e in entities)
    has_book_entity = any(e["type"] == "BOOK" for e in entities)

    # 시나리오 1: 작가 엔티티만 있고, 모델이 '도서검색'으로 예측한 경우 -> '작가검색'으로 보정
    if original_model_intent == "도서검색" and has_author_entity and not has_book_entity:
        current_intent = "작가검색" 

    # 시나리오 2: 책 엔티티만 있고, 모델이 '작가검색'으로 예측한 경우 -> '도서검색'으로 보정
    elif original_model_intent == "작가검색" and has_book_entity and not has_author_entity:
        current_intent = "도서검색"
        
    if original_model_intent != current_intent:
        print(f"INFO: Intent corrected by postprocessing. Original: '{original_model_intent}', Corrected: '{current_intent}'. Text: '{text}'")

    return {
        "intent": current_intent,
        "entities": entities,
        "original_intent": original_model_intent,
        "text": text  # 함수의 인자로 받은 text를 사용
    }

# 개선된 추론 실행
print("\n개선된 추론 예시:")
sample_texts = [
    "스즈미야 하루히의 우울 도서관에에 있나요?",
    "전생했더니 슬라임이였던 건에 대하여 책 있어?",
    "해리포터와 불의 잔 어디 있어?",
    "박완서 작가에 대해 알려줘"
]

for sample_text in sample_texts:
    print(f"\n입력 텍스트: {sample_text}")
    
    # 원본 예측
    raw_prediction = predict_intent_and_entities(
        sample_text, model, tokenizer, id2intent, id2tag
    )
    
    # 후처리로 예측 개선
    prediction = postprocess_prediction(sample_text, raw_prediction)
    
    print(f"예측 결과:")
    print(f"  의도: {prediction['intent']}")
    print(f"  개체:")
    if not prediction['entities']:
        print("    - 개체 없음")
    else:
        for entity in prediction['entities']:
            print(f"    - {entity['type']}: '{entity['text']}' (위치: {entity['start']}-{entity['end']})")

🔄 훈련 진행 중... 에포크 6/10

현재 훈련 지표:
에포크 스텝    훈련 손실    검증 손실  의도 정확도  의도 F1 점수  개체인식 F1 점수    종합 점수
  1 13 2.044167 1.133590     1.0       1.0    0.000000 0.500000
  2 26 0.765911 0.413709     1.0       1.0    0.000000 0.500000
  3 39 0.361759 0.229863     1.0       1.0    0.000000 0.500000
  4 52 0.216557 0.141529     1.0       1.0    0.761905 0.880952
  5 65 0.145732 0.103934     1.0       1.0    0.575000 0.787500
  6 78 0.108834 0.077331     1.0       1.0    0.761905 0.880952
